## Display the xml element tree

In [ ]:
from pptlayout.extractors.run_extractors import run_extractors
from json import dumps
# Replace with your actual PPTX file path
pptx_path = "/data/tianyuhu/PPTLayout/data/pptx/ZK7FNUZ33GBBCG7CFVYS56TQCTD72CJR.pptx"

# Run extractors
info = run_extractors(pptx_path, "emu")
print(dumps(info, indent=4))

In [ ]:
from pptlayout.llm.call_ollama import generate_slide_layout_suggestions
from pptlayout.llm.prompts import build_slide_layout_suggestion_prompts
from tqdm import tqdm
import os
# Call OLLAMA
output_dir = os.path.join(os.curdir, "test_output")
suggestions_list = []
for slide_info in tqdm(info["slides"]):
    slide_id = slide_info["slide_id"]
    slide_output_dir = os.path.join(output_dir, f"{slide_id}")
    if os.path.exists(os.path.join(slide_output_dir, "suggestions.txt")):
        with open(os.path.join(slide_output_dir, "suggestions.txt"), "r") as f:
            suggestions = f.read()
        suggestions_list.append(suggestions)
        continue
    prompt = build_slide_layout_suggestion_prompts(
        json_input=slide_info, 
        slide_width=info["slide_width"],
        slide_height=info["slide_height"]
        )
    suggestions = generate_slide_layout_suggestions(
        model_name="llama3.1:70b",
        prompt=prompt,
        temperature=0.5,
        )
    suggestions_list.append(suggestions)
    # save
    os.makedirs(slide_output_dir, exist_ok=True)
    with open(os.path.join(slide_output_dir, "suggestions.txt"), "w") as f:
        f.write(suggestions)

In [ ]:
from pptlayout.llm.parser import extract_json
json_list = []
for suggestions in suggestions_list:
    json_data = extract_json(suggestions)
    json_list.append(json_data)

In [ ]:
from json import dumps

# print(dumps(json_list[12], indent=4))
for json in json_list:
    print(json)

In [ ]:
from pptlayout.visualizers.layout_visualizer import generate_comparison_grid

slides = info["slides"]
# Generate comparison grid
comparison_grid = generate_comparison_grid(
    slides,
    json_list,
    slide_width=info["slide_width"],
    slide_height=info["slide_height"]
    )

## test

In [ ]:
from pptlayout.extractors.run_extractors import run_extractors
from json import dumps
# Replace with your actual PPTX file path
pptx_path = "/data/tianyuhu/PPTLayout/data/pptx/ZK7FNUZ33GBBCG7CFVYS56TQCTD72CJR.pptx"

# Run extractors
info = run_extractors(pptx_path, "emu")
# print(dumps(info, indent=4))


from pptlayout.llm.call_ollama import generate_slide_layout_suggestions
from pptlayout.llm.prompts import build_slide_layout_suggestion_prompts
test_image = "/data/tianyuhu/PPTLayout/notebooks/test_input/image.png"
# prompt = build_slide_layout_suggestion_prompts(
#     json_input=info["slides"][4],
#     slide_width=info["slide_width"],
#     slide_height=info["slide_height"],
#     )
prompt = "What can be improved just on the slide layout for a academic presentation?"
response = generate_slide_layout_suggestions(
    model_name="llama3.2-vision:11b",
    prompt=prompt,
    images=[test_image],
    ) 
print(response)

The provided image is a page from an academic presentation that discusses the design and testing of a baffle for use in nuclear materials irradiation experiments. While the content appears to be well-organized and informative, there are some aspects of the slide layout that could be improved to enhance its overall effectiveness.

*   **Image Size**: The image takes up most of the space on this page with no border around it, making the background text hard to read. 
*   **Text Spacing**: There is not enough white space between the title and photo of the baffle, the paragraph about the existing NuMI baffle, and the section explaining how they will modify that design for their baffle. The font size in the last paragraph could also be increased to make the text easier to read.
*   **Alignment**: All three paragraphs are centered on this page, but it would likely be better if there was some white space or a thin line separating them from one another.

By making these adjustments to the layo

In [ ]:
from pptlayout.llm.parser import extract_json
from json import dumps
# Extract JSON data
json_data = extract_json(response)
print(dumps(json_data, indent=4))

In [ ]:
from pptlayout.visualizers.layout_visualizer import generate_comparison_grid

generate_comparison_grid(
    [info["slides"][4]],
    [json_data],
    slide_width=info["slide_width"],
    slide_height=info["slide_height"]
    )
    